In [1]:
%load_ext autoreload
%autoreload 2

import collections
import time

import numpy as np
import pandas as pd
import pytest
import tqdm

import hetmech.degree_weight
import hetmech.hetmat

In [2]:
# Three hetmats so that there is no cache sharing
hetmat = hetmech.hetmat.HetMat('../data/hetionet-v1.0.hetmat/')
hetmat_rec = hetmech.hetmat.HetMat('../data/hetionet-v1.0.hetmat/')
hetmat_chain = hetmech.hetmat.HetMat('../data/hetionet-v1.0.hetmat/')

In [3]:
# Rephetio metapaths
metapaths = list()
for metapath in hetmat.metagraph.extract_metapaths('Compound', 'Disease', max_length=4):
    if hetmech.degree_weight.categorize(metapath) in {'long_repeat', 'other'}:
        continue
    metapaths.append(metapath)
len(metapaths)

1172

In [4]:
def equal_outputs(metapath, dense_threshold=0, dtype=np.float64):
    """Compute DWWC using all three functions, and ensure that they give the same output."""
    time1 = time.time()
    row_rec, col_rec, dwwc_rec = hetmech.degree_weight.dwwc_recursive(hetmat_rec, metapath, dense_threshold=dense_threshold, dtype=dtype)
    rec_time = time.time()
    row_chain, col_chain, dwwc_chain = hetmech.degree_weight.dwwc_chain(hetmat_chain, metapath, dense_threshold=dense_threshold, dtype=dtype)
    chain_time = time.time()
    row_original, col_original, dwwc_original = hetmech.degree_weight.dwwc(hetmat, metapath, dense_threshold=dense_threshold, dtype=dtype)
    original_time = time.time()
    times = [rec_time - time1, chain_time - rec_time, original_time - chain_time]
    try:
        assert (row_rec == row_original) and (row_chain == row_original) and (col_rec == col_original) and (col_chain == col_original)
        assert abs(dwwc_rec - dwwc_original).max() == pytest.approx(0, rel=1e-7)
        assert abs(dwwc_chain - dwwc_original).max() == pytest.approx(0, abs=1e-7)
    except AssertionError:
        print(metapath)
    return times

In [5]:
all_times = []

for metapath in tqdm.tqdm(metapaths):
    times = equal_outputs(metapath, dense_threshold=1)
    all_times.append(times)

100%|██████████| 1172/1172 [06:26<00:00,  3.03it/s]


In [6]:
df = pd.DataFrame(all_times, columns=['recursive', 'chain', 'original'])

In [7]:
df.head()

,recursive,chain,original
0,0.013190,0.010584,0.010764
1,0.006179,0.005388,0.004845
2,0.010336,0.009935,0.009551
3,0.032073,0.031519,0.030481
4,0.019795,0.017839,0.018040


In [8]:
df.mean()

recursive    0.079987
chain        0.060492
original     0.186954
dtype: float64

In [9]:
df.sum()

recursive     93.744713
chain         70.896228
original     219.110172
dtype: float64

In [10]:
from scipy import stats

In [11]:
stats.ttest_rel(df['recursive'], df['original'])

Ttest_relResult(statistic=-12.849804121802832, pvalue=1.8924834621872183e-35)

In [12]:
stats.ttest_rel(df['chain'], df['original'])

Ttest_relResult(statistic=-14.954073494917703, pvalue=2.106070066895803e-46)

## Example of typically long computations

In [13]:
%%timeit

_ = hetmech.degree_weight.dwwc(hetmat, 'CbGeAeGaD', dense_threshold=0)

3.05 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit

_ = hetmech.degree_weight.dwwc(hetmat, 'CbGeAeGaD', dense_threshold=1)

2.54 s ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit

_ = hetmech.degree_weight.dwwc_recursive(hetmat, 'CbGeAeGaD', dense_threshold=0)

2.08 s ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit

_ = hetmech.degree_weight.dwwc_recursive(hetmat, 'CbGeAeGaD', dense_threshold=1)

356 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit

_ = hetmech.degree_weight.dwwc_chain(hetmat, 'CbGeAeGaD', dense_threshold=0)

2.07 s ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit

_ = hetmech.degree_weight.dwwc_chain(hetmat, 'CbGeAeGaD', dense_threshold=1)

295 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
